In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass

# ==== تنظیمات اولیه ====
input_folder = r"\\isd_netapp\ukb$\Majid\deepflow\deepFlowDocker\output\output1\results"
#\\isd_netapp\ukb$\Majid\deepflow\deepFlowDocker\output\output1\results
#P:/Projects/DeepFlow/deepFlowDocker/scripts/Registration/data
target_frame_index = 7
threshold = 1e-6

# ==== بارگذاری فایل ====
file_list = [f for f in os.listdir(input_folder) if f.endswith('.npy')]
first_file_path = os.path.join(input_folder, file_list[0])
velocity_array = np.load(first_file_path)
frame = velocity_array[target_frame_index]

# ==== تحلیل عددی ====
binary_mask = (np.abs(frame) > threshold).astype(np.uint8)
non_zero_count = np.count_nonzero(frame)
binary_count = np.count_nonzero(binary_mask)
com = center_of_mass(binary_mask) if binary_count > 0 else (np.nan, np.nan)

print("=" * 70)
print(f" File: {file_list[0]}")
print(f"  Frame Index: {target_frame_index}")
print("-" * 70)
print(f"Shape: {frame.shape}")
print(f"Contains NaN? {' YES' if np.isnan(frame).any() else ' NO'}")
print(f"Min / Max: {np.min(frame)} / {np.max(frame)}")
print(f"Unique values: {np.unique(frame)[:10]} ...")
print(f"Non-zero pixels: {non_zero_count} ({non_zero_count / frame.size:.4%})")
print("-" * 70)
print(f"🧪 Binary Thresholding (|value| > {threshold})")
print(f"Binary non-zero count: {binary_count} ({binary_count / frame.size:.4%})")
print(f"Center of Mass: {com}")
print("=" * 70)

# ==== نمایش تصویری ====
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle(f"Frame {target_frame_index} – Velocity and Binary Mask", fontsize=14)

# --- تصویر velocity با نوار رنگی ---
abs_max = np.max(np.abs(frame))
im0 = axs[0].imshow(frame, cmap='seismic', vmin=-abs_max, vmax=abs_max)
axs[0].set_title("Velocity Frame")
axs[0].axis('off')

# اضافه کردن colorbar
cbar = fig.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)
cbar.set_label("Velocity", fontsize=10)

# --- تصویر ماسک باینری ---
axs[1].imshow(binary_mask, cmap='gray')
axs[1].set_title(f"Binary Mask (>{threshold})")
axs[1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Visualize multiple VENC velocity frames, masks, overlay contours, and zoomed CINE frame
# Author: ChatGPT – customized for user

import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pydicom
import tempfile
from scipy.ndimage import center_of_mass
from scipy.spatial.distance import cdist

# === Parameters ===
velocity_folder = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\1007337_v2_20213_2_0_masked.npy"
mask_folder = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\UnetOutput"
zip_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\raw\1007337_v2_20213_2_0.zip"
thresh = 1e-6
venc = 250
num_frames_to_visualize = 29  # Change this to 1, 5, 10, etc.

# === Load velocity file ===
#velocity_files = [f for f in os.listdir(velocity_folder) if f.endswith('.npy')]
#velocity_path = os.path.join(velocity_folder, velocity_files[0])
velocity_array = np.load(velocity_folder)

# === Load raw model masks ===
debug_mask_path = os.path.join(mask_folder, "debug_raw_masks_1003775_v2.npy")
raw_masks = np.load(debug_mask_path)

# === Extract DICOMs ===
temp_extract_path = tempfile.mkdtemp()
with zipfile.ZipFile(zip_path, 'r') as archive:
    archive.extractall(temp_extract_path)

venc_frames = []
cine_frames = []

for root, _, files in os.walk(temp_extract_path):
    for file in files:
        if file.endswith('.dcm'):
            file_path = os.path.join(root, file)
            try:
                ds = pydicom.dcmread(file_path, force=True)
                description = str(ds[0x0008, 0x103E].value)
                if '@c_P' in description:
                    venc_frames.append((ds.InstanceNumber, file_path))
                elif description.split('@')[-1] == 'c':
                    cine_frames.append((ds.InstanceNumber, file_path))
            except:
                continue

venc_frames.sort(key=lambda x: x[0])
cine_frames.sort(key=lambda x: x[0])

# === Process N frames ===
for target_frame_index in range(num_frames_to_visualize):
    frame = velocity_array[target_frame_index]
    binary_mask = (np.abs(frame) > thresh).astype(np.uint8)
    raw_mask = raw_masks[target_frame_index]
    raw_mask_rotated = cv2.rotate(raw_mask, cv2.ROTATE_90_CLOCKWISE)
    raw_mask_rotated = cv2.resize(raw_mask_rotated, (192, 192), interpolation=cv2.INTER_AREA)
    raw_mask_binary = (raw_mask_rotated > 0.5).astype(np.uint8)

    diff_mask = np.abs(binary_mask - raw_mask_binary)
    diff_pixels = np.count_nonzero(diff_mask)
    match_pixels = np.count_nonzero(binary_mask == raw_mask_binary)
    total_pixels = binary_mask.size
    match_ratio = match_pixels / total_pixels * 100
    raw_only_mask = (raw_mask_binary == 1) & (binary_mask == 0)
    raw_only_count = np.count_nonzero(raw_only_mask)

    selected_path = venc_frames[target_frame_index][1]
    selected_ds = pydicom.dcmread(selected_path)
    rescale_slope = float(getattr(selected_ds, 'RescaleSlope', 1.0))
    rescale_intercept = float(getattr(selected_ds, 'RescaleIntercept', 0.0))
    pixel_array = selected_ds.pixel_array.astype(np.float32)
    pixel_array = pixel_array * rescale_slope + rescale_intercept
    pixel_array = cv2.rotate(pixel_array, cv2.ROTATE_90_CLOCKWISE)
    velocity_image = pixel_array * (venc / 4096)

    contours, _ = cv2.findContours(raw_mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    print("=" * 80)
    #print(f" File: {velocity_files[0]} | Frame: {target_frame_index}")
    print(f" File: {os.path.basename(velocity_folder)} | Frame: {target_frame_index}")
    print("-" * 80)
    print(f"Velocity Min/Max: {np.min(frame):.2f} / {np.max(frame):.2f}")
    print(f"Velocity Non-zero count: {np.count_nonzero(frame)}")
    print(f"Binary Mask Non-zero: {np.count_nonzero(binary_mask)}")
    print(f"Rotated Raw Mask Non-zero: {np.count_nonzero(raw_mask_binary)}")
    print(f"Different pixels: {diff_pixels} ({diff_pixels / total_pixels:.4%})")
    print(f"Matching pixels: {match_pixels} ({match_ratio:.2f}%)")
    print("=" * 80)
    print(f" Pixels in Raw Mask ONLY (missed by velocity mask): {raw_only_count}")
    print("=" * 80)

    print("=" * 60)
    print(f"🔎 Intensity statistics of velocity image:")
    print(f"  Min value     : {np.min(velocity_image):.2f}")
    print(f"  Max value     : {np.max(velocity_image):.2f}")
    print(f"  Mean value    : {np.mean(velocity_image):.2f}")
    print(f"  Median value  : {np.median(velocity_image):.2f}")
    print(f"  Std deviation : {np.std(velocity_image):.2f}")
    print("=" * 60)

    com = center_of_mass(raw_mask_binary)
    pts = contours[0][:, 0, :]
    distances = cdist([com], pts)[0]
    mean_radius = np.mean(distances)
    pad = max(int(mean_radius * 1.1), 30)

    center_x, center_y = int(com[1]), int(com[0])
    x1 = max(center_x - pad, 0)
    x2 = min(center_x + pad, velocity_image.shape[1])
    y1 = max(center_y - pad, 0)
    y2 = min(center_y + pad, velocity_image.shape[0])
    zoomed_crop = velocity_image[y1:y2, x1:x2]
    zoomed_mask = raw_mask_binary[y1:y2, x1:x2]

    cine_img = None
    cine_zoomed = None
    if target_frame_index < len(cine_frames):
        cine_ds = pydicom.dcmread(cine_frames[target_frame_index][1])
        cine_img = cv2.rotate(cine_ds.pixel_array.astype(np.float32), cv2.ROTATE_90_CLOCKWISE)
        cine_zoomed = cine_img[y1:y2, x1:x2]

    fig, axs = plt.subplots(3, 3, figsize=(20, 12))
    fig.suptitle(f"Frame {target_frame_index} – Velocity Mask vs. Raw Model Mask", fontsize=16)

    abs_max = np.max(np.abs(frame))
    im0 = axs[0, 0].imshow(frame, cmap='seismic', vmin=-abs_max, vmax=abs_max)
    axs[0, 0].set_title("Velocity Frame (.npy)")
    axs[0, 0].axis('off')
    fig.colorbar(im0, ax=axs[0, 0], fraction=0.046, pad=0.04).set_label("Velocity", fontsize=10)

    overlay = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
    overlay[binary_mask == 1] = [255, 0, 0]
    overlay[raw_mask_binary == 1] = [255, 255, 255]
    overlay[(binary_mask == 1) & (raw_mask_binary == 1)] = [0, 255, 0]
    axs[0, 1].imshow(overlay)
    axs[0, 1].set_title("Mask Comparison\nWhite: Raw | Blue: Binary | Green: Overlap")
    axs[0, 1].axis('off')

    axs[0, 2].imshow(binary_mask, cmap='gray')
    axs[0, 2].set_title("Binary Mask (Thresholded Velocity)")
    axs[0, 2].axis('off')

    axs[1, 0].imshow(diff_mask, cmap='hot')
    axs[1, 0].set_title("Difference Map (|Binary - Raw|)")
    axs[1, 0].axis('off')

    highlight_raw_only = np.zeros((*frame.shape, 3), dtype=np.uint8)
    highlight_raw_only[raw_only_mask] = [255, 0, 255]
    axs[1, 1].imshow(frame, cmap='gray')
    axs[1, 1].imshow(highlight_raw_only, alpha=0.6)
    axs[1, 1].set_title("Raw Mask Only (Magenta)")
    axs[1, 1].axis('off')

    im7 = axs[1, 2].imshow(velocity_image, cmap='seismic', vmin=np.min(velocity_image), vmax=np.max(velocity_image))
    axs[1, 2].set_title("Velocity Image (from DICOM)")
    axs[1, 2].axis('off')
    fig.colorbar(im7, ax=axs[1, 2], fraction=0.046, pad=0.04).set_label("Velocity (cm/s)", fontsize=10)

    im8 = axs[2, 2].imshow(zoomed_crop, cmap='seismic', vmin=np.min(zoomed_crop), vmax=np.max(zoomed_crop))
    axs[2, 2].set_title("Zoomed Aortic Region")
    axs[2, 2].axis('off')
    axs[2, 2].contour(zoomed_mask, colors='cyan', linewidths=1)
    fig.colorbar(im8, ax=axs[2, 2], fraction=0.046, pad=0.04).set_label("Velocity (cm/s)", fontsize=10)

    axs[2, 1].imshow(velocity_image, cmap='seismic', vmin=np.min(velocity_image), vmax=np.max(velocity_image))
    axs[2, 1].set_title("Velocity + Raw Contour")
    axs[2, 1].axis('off')
    for contour in contours:
        axs[2, 1].plot(contour[:, 0, 0], contour[:, 0, 1], color='cyan', linewidth=1)
    fig.colorbar(axs[2, 1].images[0], ax=axs[2, 1], fraction=0.046, pad=0.04).set_label("Velocity (cm/s)", fontsize=10)

    if cine_zoomed is not None:
        axs[2, 0].imshow(cine_zoomed, cmap='gray')
        axs[2, 0].contour(zoomed_mask, colors='cyan', linewidths=1)
        axs[2, 0].set_title("Zoomed CINE + Raw Contour")
        axs[2, 0].axis('off')
    else:
        axs[2, 0].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Visualize VENC velocity frames with masks, raw contours, and CINE overlays
# Author: ChatGPT – customized for user

import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pydicom
import tempfile
from scipy.ndimage import center_of_mass
from scipy.spatial.distance import cdist

# === Parameters ===
velocity_folder = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\1007337_v2_20213_2_0_masked.npy"
mask_folder = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\UnetOutput"
zip_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\raw\1007337_v2_20213_2_0.zip"
thresh = 1e-6
venc = 250
num_frames_to_visualize = 29  # You can change this to any number

# === Load velocity .npy file ===
#velocity_files = [f for f in os.listdir(velocity_folder) if f.endswith('.npy')]
#velocity_path = os.path.join(velocity_folder, velocity_files[0])
velocity_array = np.load(velocity_folder)

# === Load raw predicted masks ===
debug_mask_path = os.path.join(mask_folder, "debug_raw_masks_1007337_v2.npy")
raw_masks = np.load(debug_mask_path)

# === Extract DICOM files ===
temp_extract_path = tempfile.mkdtemp()
with zipfile.ZipFile(zip_path, 'r') as archive:
    archive.extractall(temp_extract_path)

venc_frames = []
cine_frames = []

for root, _, files in os.walk(temp_extract_path):
    for file in files:
        if file.endswith('.dcm'):
            path = os.path.join(root, file)
            try:
                ds = pydicom.dcmread(path, force=True)
                desc = str(ds[0x0008, 0x103E].value)
                if '@c_P' in desc:
                    venc_frames.append((ds.InstanceNumber, path))
                elif desc.endswith('@c') and '_P_' not in desc:
                    cine_frames.append((ds.InstanceNumber, path))
            except:
                continue

venc_frames.sort(key=lambda x: x[0])
cine_frames.sort(key=lambda x: x[0])

# === Loop over selected frames ===
for target_frame_index in range(num_frames_to_visualize):
    frame = velocity_array[target_frame_index]
    binary_mask = (np.abs(frame) > thresh).astype(np.uint8)
    raw_mask = raw_masks[target_frame_index]
    raw_mask_rotated = cv2.rotate(raw_mask, cv2.ROTATE_90_CLOCKWISE)
    raw_mask_rotated = cv2.resize(raw_mask_rotated, (192, 192), interpolation=cv2.INTER_AREA)
    raw_mask_binary = (raw_mask_rotated > 0.5).astype(np.uint8)

    diff_mask = np.abs(binary_mask - raw_mask_binary)
    diff_pixels = np.count_nonzero(diff_mask)
    match_pixels = np.count_nonzero(binary_mask == raw_mask_binary)
    total_pixels = binary_mask.size
    match_ratio = match_pixels / total_pixels * 100
    raw_only_mask = (raw_mask_binary == 1) & (binary_mask == 0)
    raw_only_count = np.count_nonzero(raw_only_mask)

    selected_path = venc_frames[target_frame_index][1]
    selected_ds = pydicom.dcmread(selected_path)
    slope = float(getattr(selected_ds, 'RescaleSlope', 1.0))
    intercept = float(getattr(selected_ds, 'RescaleIntercept', 0.0))
    pixel_array = selected_ds.pixel_array.astype(np.float32)
    pixel_array = pixel_array * slope + intercept
    pixel_array = cv2.rotate(pixel_array, cv2.ROTATE_90_CLOCKWISE)
    velocity_image = pixel_array * (venc / 4096)

    contours, _ = cv2.findContours(raw_mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # === Print statistics ===
    print("=" * 80)
    #print(f" File: {velocity_files[0]} | Frame: {target_frame_index}")
    print(f" File: {os.path.basename(velocity_folder)} | Frame: {target_frame_index}")
    print("-" * 80)
    print(f"Velocity Min/Max: {np.min(frame):.2f} / {np.max(frame):.2f}")
    print(f"Velocity Non-zero count: {np.count_nonzero(frame)}")
    print(f"Binary Mask Non-zero: {np.count_nonzero(binary_mask)}")
    print(f"Rotated Raw Mask Non-zero: {np.count_nonzero(raw_mask_binary)}")
    print(f"Different pixels: {diff_pixels} ({diff_pixels / total_pixels:.4%})")
    print(f"Matching pixels: {match_pixels} ({match_ratio:.2f}%)")
    print("=" * 80)
    print(f" Pixels in Raw Mask ONLY (missed by velocity mask): {raw_only_count}")
    print("=" * 80)

    print("=" * 60)
    print(f"🔎 Intensity statistics of velocity image:")
    print(f"  Min value     : {np.min(velocity_image):.2f}")
    print(f"  Max value     : {np.max(velocity_image):.2f}")
    print(f"  Mean value    : {np.mean(velocity_image):.2f}")
    print(f"  Median value  : {np.median(velocity_image):.2f}")
    print(f"  Std deviation : {np.std(velocity_image):.2f}")
    print("=" * 60)

    # === Compute aortic bounding box ===
    com = center_of_mass(raw_mask_binary)
    pts = contours[0][:, 0, :]
    distances = cdist([com], pts)[0]
    mean_radius = np.mean(distances)
    #===============
    print(f" Frame {target_frame_index}: mean_radius = {mean_radius:.2f} pixels")
    #===============
    pad = max(int(mean_radius * 1.1), 30)
    cx, cy = int(com[1]), int(com[0])
    x1 = max(cx - pad, 0)
    x2 = min(cx + pad, velocity_image.shape[1])
    y1 = max(cy - pad, 0)
    y2 = min(cy + pad, velocity_image.shape[0])
    zoomed_crop = velocity_image[y1:y2, x1:x2]
    zoomed_mask = raw_mask_binary[y1:y2, x1:x2]
    
    # === Load full and zoomed cine ===
    cine_full = cine_zoomed = None
    if target_frame_index < len(cine_frames):
        cine_ds = pydicom.dcmread(cine_frames[target_frame_index][1])
        cine_img = cv2.rotate(cine_ds.pixel_array.astype(np.float32), cv2.ROTATE_90_CLOCKWISE)
        cine_full = cine_img
        cine_zoomed = cine_img[y1:y2, x1:x2]

    # === Plotting ===
    fig, axs = plt.subplots(3, 3, figsize=(20, 12))
    fig.suptitle(f"Frame {target_frame_index} – Velocity Mask vs. Raw Model Mask", fontsize=16)

    abs_max = np.max(np.abs(frame))
    im0 = axs[0, 0].imshow(frame, cmap='seismic', vmin=-abs_max, vmax=abs_max)
    axs[0, 0].set_title("Velocity Frame (.npy)")
    axs[0, 0].axis('off')
    fig.colorbar(im0, ax=axs[0, 0], fraction=0.046, pad=0.04).set_label("Velocity", fontsize=10)

    overlay = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
    overlay[binary_mask == 1] = [255, 0, 0]
    overlay[raw_mask_binary == 1] = [255, 255, 255]
    overlay[(binary_mask == 1) & (raw_mask_binary == 1)] = [0, 255, 0]
    axs[0, 1].imshow(overlay)
    axs[0, 1].set_title("Mask Comparison\nWhite: Raw | Blue: Binary | Green: Overlap")
    axs[0, 1].axis('off')

    axs[0, 2].imshow(binary_mask, cmap='gray')
    axs[0, 2].set_title("Binary Mask (Thresholded Velocity)")
    axs[0, 2].axis('off')

    # === Image 4: Full CINE with raw contour ===
    if cine_full is not None:
        axs[1, 0].imshow(cine_full, cmap='gray')
        axs[1, 0].contour(raw_mask_binary, colors='cyan', linewidths=1)
        axs[1, 0].set_title("Full CINE + Raw Contour")
    else:
        axs[1, 0].set_title("CINE Not Available")
    axs[1, 0].axis('off')

    highlight_raw_only = np.zeros((*frame.shape, 3), dtype=np.uint8)
    highlight_raw_only[raw_only_mask] = [255, 0, 255]
    axs[1, 1].imshow(frame, cmap='gray')
    axs[1, 1].imshow(highlight_raw_only, alpha=0.6)
    axs[1, 1].set_title("Raw Mask Only (Magenta)")
    axs[1, 1].axis('off')

    im7 = axs[1, 2].imshow(velocity_image, cmap='seismic', vmin=np.min(velocity_image), vmax=np.max(velocity_image))
    axs[1, 2].set_title("Velocity Image (from DICOM)")
    axs[1, 2].axis('off')
    fig.colorbar(im7, ax=axs[1, 2], fraction=0.046, pad=0.04).set_label("Velocity (cm/s)", fontsize=10)

    # === Image 8: Zoomed aortic ===
    im8 = axs[2, 2].imshow(zoomed_crop, cmap='seismic', vmin=np.min(zoomed_crop), vmax=np.max(zoomed_crop))
    axs[2, 2].set_title("Zoomed Aortic Region")
    axs[2, 2].axis('off')
    axs[2, 2].contour(zoomed_mask, colors='cyan', linewidths=1)
    fig.colorbar(im8, ax=axs[2, 2], fraction=0.046, pad=0.04).set_label("Velocity (cm/s)", fontsize=10)

    axs[2, 1].imshow(velocity_image, cmap='seismic', vmin=np.min(velocity_image), vmax=np.max(velocity_image))
    axs[2, 1].set_title("Velocity + Raw Contour")
    axs[2, 1].axis('off')
    for contour in contours:
        axs[2, 1].plot(contour[:, 0, 0], contour[:, 0, 1], color='cyan', linewidth=1)
    fig.colorbar(axs[2, 1].images[0], ax=axs[2, 1], fraction=0.046, pad=0.04).set_label("Velocity (cm/s)", fontsize=10)

    # === Image 9: Zoomed CINE ===
    if cine_zoomed is not None:
        axs[2, 0].imshow(cine_zoomed, cmap='gray')
        axs[2, 0].contour(zoomed_mask, colors='cyan', linewidths=1)
        axs[2, 0].set_title("Zoomed CINE + Raw Contour")
    else:
        axs[2, 0].set_title("Zoomed CINE Not Available")
    axs[2, 0].axis('off')

    plt.tight_layout()
    plt.show()
